# Seznam kandidátů
Budeme to brát z DBF souborů na stránce [otevřených dat ČSÚ](http://www.volby.cz/opendata/opendata.htm) - konkrétně "Registry". K některým máme XML a někde XLSX. Zpracování tu je málo, jen se párujou lidi a strany.

In [3]:
import dbfread
from simpledbf import Dbf5
import pandas as pd
import xmltodict
from zipfile import ZipFile
from glob import glob
import os
%matplotlib inline

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [4]:
dr = 'vystupy/'

## PSP
DBF

In [3]:
for fn in glob('data/PS*.zip'):
    fnk = os.path.split(fn)[-1].split('.')[0] + '.csv'
    with ZipFile(fn) as zf:
        df = Dbf5(zf.extract('PSRK.dbf'), codec='cp852').to_dataframe()
        os.remove('PSRK.dbf')

        df2 = Dbf5(zf.extract('PSRKL.dbf'), codec='cp852').to_dataframe()
        os.remove('PSRKL.dbf')

        df.merge(df2, on='KSTRANA').to_csv(os.path.join(dr, fnk))

## Kraje
DBF

In [4]:
for fn in glob('data/kz*.zip'):
    fnk = os.path.split(fn)[-1].split('.')[0] + '.csv'
    with ZipFile(fn) as zf:
        kz = [j.filename for j in zf.filelist if j.filename.startswith('KZRK.')][0]
        df = Dbf5(zf.extract(kz), codec='cp852').to_dataframe()
        os.remove(kz)

        kz = [j.filename for j in zf.filelist if j.filename.startswith('KZRKL.')][0]
        df2 = Dbf5(zf.extract(kz), codec='cp852').to_dataframe()
        os.remove(kz)

        df.join(df2, on='KSTRANA', how='left', rsuffix='str_').to_csv(os.path.join(dr, fnk))

## EP
XML

In [5]:
for fn in glob('data/EP*.zip'):
    fnk = fn.split('/')[-1].replace('.zip', '.csv')
    with ZipFile(fn) as zf:
        fns = [j.filename for j in zf.filelist]
        if len(fns) != 2 or 'eprkl.xml' not in fns or 'eprk.xml' not in fns:
            raise ValueError('err')

        with zf.open('eprkl.xml') as f:
            strany = pd.DataFrame(xmltodict.parse(f.read())['EP_RKL']['EP_RKL_ROW'])
        with zf.open('eprk.xml') as f:
            kandi = pd.DataFrame(xmltodict.parse(f.read())['EP_REGKAND']['EP_REGKAND_ROW'])
            
        kandi.merge(strany, on='ESTRANA').to_csv(os.path.join(dr, fnk))

## Senát
XLSX

In [6]:
for fn in glob('data/SE*'):
    fnk = os.path.split(fn)[-1].split('.')[0] + '.csv'
    with ZipFile(fn) as zf:
        pd.read_excel(zf.open('serk.xlsx')).to_csv(os.path.join(dr, fnk))

In [ ]:
df=pd.read_excel('data/KV2014reg20141014_xlsx/kvrk.xlsx')

In [ ]:
df.head(1).T